In [1]:
import matplotlib.pyplot as plt
import random
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np

In [2]:
df = pd.read_csv('train.csv', index_col=0)
df.head()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
Id,,,,,,,,,,,,
0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  10000 non-null  float64
 1   years_of_experience  10000 non-null  float64
 2   lesson_price         10000 non-null  float64
 3   qualification        10000 non-null  float64
 4   physics              10000 non-null  float64
 5   chemistry            10000 non-null  float64
 6   biology              10000 non-null  float64
 7   english              10000 non-null  float64
 8   geography            10000 non-null  float64
 9   history              10000 non-null  float64
 10  mean_exam_points     10000 non-null  float64
 11  choose               10000 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 1015.6 KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,10000.0,45.8009,8.030274,23.0,40.0,46.0,51.0,68.0
years_of_experience,10000.0,1.9748,1.766883,0.0,0.0,2.0,3.0,9.0
lesson_price,10000.0,1702.4400,523.789062,200.0,1300.0,1550.0,2150.0,3950.0
qualification,10000.0,1.7243,0.798845,1.0,1.0,2.0,2.0,4.0
physics,10000.0,0.3706,0.482990,0.0,0.0,0.0,1.0,1.0
chemistry,10000.0,0.1215,0.326724,0.0,0.0,0.0,0.0,1.0
biology,10000.0,0.1172,0.321675,0.0,0.0,0.0,0.0,1.0
english,10000.0,0.0591,0.235824,0.0,0.0,0.0,0.0,1.0
geography,10000.0,0.0277,0.164120,0.0,0.0,0.0,0.0,1.0
history,10000.0,0.0180,0.132958,0.0,0.0,0.0,0.0,1.0


Попробуем добвать новые параметры.
Сгруппируем параметры lesson_price, years_of_experience, mean_exam_points по группам.

In [5]:
exam_points_group_treshholds = [df['mean_exam_points'].min(), 
                                df['mean_exam_points'].quantile(0.25),
                                df['mean_exam_points'].quantile(0.50),
                                df['mean_exam_points'].quantile(0.75),
                                df['mean_exam_points'].max()]
df['exam_points_groups'] = pd.cut(df['mean_exam_points'],exam_points_group_treshholds,labels=[1,2,3,4],include_lowest = True).astype('float')

In [6]:
experience_group_treshholds =  [df['years_of_experience'].min(), 
                                df['years_of_experience'].quantile(0.50),
                                df['years_of_experience'].quantile(0.75),
                                df['years_of_experience'].max()]
df['years_of_experience_groups'] = pd.cut(df['years_of_experience'],experience_group_treshholds,labels=[1,2,3],include_lowest = True).astype('float')

In [7]:
lesson_price_group_treshholds = [df['lesson_price'].min(), 
                                df['lesson_price'].quantile(0.25),
                                df['lesson_price'].quantile(0.50),
                                df['lesson_price'].quantile(0.75),
                                df['lesson_price'].max()]
df['lesson_price_groups'] = pd.cut(df['lesson_price'],lesson_price_group_treshholds,labels=[1,2,3,4],include_lowest = True).astype('float')

Также создадим новые параметры, которые будут указывать то, к каким наукам идет уклон, к естественным или гуманитраным.

In [8]:
df['sciences'] = np.where((df['physics'] + df['chemistry'] + df['biology']) >= 2, 1, 0)

In [9]:
df['humanities'] = np.where((df['english'] + df['geography'] + df['history']) >= 2, 1, 0)

In [10]:
classification_data = df.drop('choose', axis = 1).to_numpy()
classification_labels = df['choose'].to_numpy()

In [11]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [12]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data # значения признаков
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его   
        prediction = max(classes, key=classes.get)
        return prediction

In [13]:
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [14]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels) # Функционал качества

In [15]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [16]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels) 

    best_quality = 0
    best_t = None # лучший порог разбиения
    best_index = None # лучший индекс разбиения
    
    n_features = data.shape[1] # кол-во признаков
    
    for index in range(n_features): # проход по всем признакам
        t_values = [row[index] for row in data] # берем столбец/признак с соотв. индексом
        
        for t in t_values: # проход по признаку
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t) # делаем разбиение
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue # начинаем следующий проход цикла, минуя оставшееся тело цикла
            
            # расчет качества текущего разбиения
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [17]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels) # ищем лучшее разбиение
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    # неопределенность после разбиения осталась такой же как до
    if quality == 0: # критерий останова
#         print('leaf')
        return Leaf(data, labels) # считаем прогноз для листьев

    # если качество улучшилось, то делим дерево по лучшему разбиению
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [18]:
# Проход объекта по дереву для его классификации

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf): # проверка текущий узел это лист?
        answer = node.prediction # считаем прогноз для листа
        return answer

    if obj[node.index] <= node.t: # если значение признака меньше порога t
        return classify_object(obj, node.true_branch) # рекурсия: отправляем объект в true-ветку
    else:
        return classify_object(obj, node.false_branch) # рекурсия: отправляем объект в false-ветку

In [19]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree) # определяем ветки для объектов
        classes.append(prediction)
    return classes

In [20]:
# Построим дерево по обучающей выборке
my_tree = build_tree(classification_data, classification_labels)

In [21]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [22]:
# Точность на обучающей выборке
answers = predict(classification_data, my_tree)
train_accuracy = accuracy_metric(classification_labels, answers)
train_accuracy

92.27

In [23]:
test = pd.read_csv('test.csv')
X_submission = test[test.columns[1:]].copy()

In [24]:
exam_points_group_treshholds = [X_submission['mean_exam_points'].min(), 
                                X_submission['mean_exam_points'].quantile(0.25),
                                X_submission['mean_exam_points'].quantile(0.50),
                                X_submission['mean_exam_points'].quantile(0.75),
                                X_submission['mean_exam_points'].max()]
X_submission['exam_points_groups'] = pd.cut(X_submission['mean_exam_points'],exam_points_group_treshholds,labels=[1,2,3,4],include_lowest = True).astype('float')

experience_group_treshholds =  [X_submission['years_of_experience'].min(), 
                                X_submission['years_of_experience'].quantile(0.50),
                                X_submission['years_of_experience'].quantile(0.75),
                                X_submission['years_of_experience'].max()]
df['years_of_experience_groups'] = pd.cut(X_submission['years_of_experience'],experience_group_treshholds,labels=[1,2,3],include_lowest = True).astype('float')


lesson_price_group_treshholds = [X_submission['lesson_price'].min(), 
                                X_submission['lesson_price'].quantile(0.25),
                                X_submission['lesson_price'].quantile(0.50),
                                X_submission['lesson_price'].quantile(0.75),
                                X_submission['lesson_price'].max()]
X_submission['lesson_price_groups'] = pd.cut(X_submission['lesson_price'],lesson_price_group_treshholds,labels=[1,2,3,4],include_lowest = True).astype('float')

X_submission['sciences'] = np.where((X_submission['physics'] + X_submission['chemistry'] + X_submission['biology']) >= 2, 1, 0)

X_submission['humanities'] = np.where((X_submission['english'] + X_submission['geography'] + X_submission['history']) >= 2, 1, 0)

In [25]:
X_submission = X_submission.to_numpy()

In [26]:
y_submission = predict(X_submission, my_tree)

In [27]:
test['choose'] = y_submission
submission = test[['Id','choose']]
submission.choose.value_counts()

0    9321
1     679
Name: choose, dtype: int64

In [28]:
submission.to_csv('submission_pavellisichkin.csv', index=False)

**Your Best Entry 
Your submission scored 0.61316, which is an improvement of your previous score of 0.54839.**

Я пробовал также сделать классификацию через лес и adaboost, но они давали результат хуже этого.
Можно было бы попробовать различные глубины для дерева, но реализовать останов по глубине у меня не вышло.
Так что пока это лучший реультат которого я добился.
С новыми параметрами тоже не особо понимаю правильно ли я их создавал и вообще как создаются новые параметрами пока не особо понятно.